In [1]:
import os
import subprocess
import numpy as np
import pandas as pd
info = pd.read_excel("data/NanoNASCseq_summary_selected.xls")
strains = ["K562", "mESC"]

# Gene expression

In [2]:
for strain in strains:
    for group in ["all_cells", "control_cells", "treatment_cells"]:
        d = info
        d = d[(d["Strain"] == strain) & (d["Time"] == 3) & (d["ActD"].isna()) & (d["UMIs"] >= 5000)]
        
        if strain == "K562":
            if group == "all_cells":
                d = d[(d["s4U"] == 0) | (d["s4U"] == 50)]
            elif group == "control_cells":
                d = d[(d["s4U"] == 0)]
            elif group == "treatment_cells":
                d = d[(d["TCRatio"] >= 0.008) & (d["s4U"] == 50)]
            else:
                assert False
            anno_path = "/home/chenzonggui/species/homo_sapiens/GRCh38.p13/gencode.v39.genes.tsv"
        elif strain == "mESC":
            if group == "all_cells":
                d = d[(d["s4U"] == 0) | (d["s4U"] == 400)]
            elif group == "control_cells":
                d = d[(d["s4U"] == 0)]
            elif group == "treatment_cells":
                d = d[(d["TCRatio"] >= 0.01) & (d["s4U"] == 400)]
            else:
                assert False
            anno_path = "/home/chenzonggui/species/mus_musculus/GRCm38.p6/gencode.vM25.genes.tsv"
        else:
            assert False
            
        anno = pd.read_csv(anno_path, sep="\t", index_col=2)
        
        print(strain, group, len(d), sep="\t")

        for tc in [1, 2]:
            outfile1 = "results/expression/report/gene_based/%s.%s.min_read_2_min_tc_%d.filelist.txt" % (strain, group, tc)
            outfile2 = "results/expression/report/gene_based/%s.%s.min_read_2_min_tc_%d.tsv" % (strain, group, tc)
            outfile3 = "results/expression/report/gene_based/%s.%s.min_read_2_min_tc_%d.annotated.tsv" % (strain, group, tc)

            with open(outfile1, "w+") as fw:
                for run, cell in d[["Run", "Cell"]].values:
                    path = "results/expression/quant_genes/min_read_2_min_tc_%d/%s/%s.tsv" % (tc, run, cell)
                    fw.write(path + "\n")
            cmd = "./scripts/merge_counts.py %s %s" % (outfile1, outfile2)
            subprocess.check_call(cmd, shell=True)
            dat = pd.read_csv(outfile2, sep="\t", header=0, index_col=0)
            dat["NTR"] = dat["Nascent"] / dat["Total"]
            dat["TPM"] = dat["Total"] * 1e6 / sum(dat["Total"])
            dat["T"] = -3/np.log2(1-dat["NTR"])
            dat = dat.merge(anno, left_index=True, right_index=True, how="left")
            dat.to_csv(outfile3, sep="\t")
            os.remove(outfile1)
            os.remove(outfile2)

K562	all_cells	369


/home/chenzonggui/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


K562	control_cells	190


/home/chenzonggui/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


K562	treatment_cells	105


...

/home/chenzonggui/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


mESC	control_cells	64


/home/chenzonggui/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


mESC	treatment_cells	110


/home/chenzonggui/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


# Isoform expression

In [3]:
for strain in strains:
    for group in ["all_cells", "control_cells", "treatment_cells"]:
        d = info
        d = d[(d["Strain"] == strain) & (d["Time"] == 3) & (d["ActD"].isna()) & (d["UMIs"] >= 5000)]
        
        if strain == "K562":
            if group == "all_cells":
                d = d[(d["s4U"] == 0) | (d["s4U"] == 50)]
            elif group == "control_cells":
                d = d[(d["s4U"] == 0)]
            elif group == "treatment_cells":
                d = d[(d["TCRatio"] >= 0.008) & (d["s4U"] == 50)]
            else:
                assert False
            anno_path = "/home/chenzonggui/species/homo_sapiens/GRCh38.p13/gencode.v39.annotation.tsv"
        elif strain == "mESC":
            if group == "all_cells":
                d = d[(d["s4U"] == 0) | (d["s4U"] == 400)]
            elif group == "control_cells":
                d = d[(d["s4U"] == 0)]
            elif group == "treatment_cells":
                d = d[(d["TCRatio"] >= 0.01) & (d["s4U"] == 400)]
            else:
                assert False
            anno_path = "/home/chenzonggui/species/mus_musculus/GRCm38.p6/gencode.vM25.annotation.tsv"
        else:
            assert False
            
        anno = pd.read_csv(anno_path, sep="\t", index_col=0)
        
        print(strain, group, len(d), sep="\t")

        for tc in [1, 2]:
            outfile1 = "results/expression/report/isoform_based/%s.%s.min_read_2_min_tc_%d.filelist.txt" % (strain, group, tc)
            outfile2 = "results/expression/report/isoform_based/%s.%s.min_read_2_min_tc_%d.tsv" % (strain, group, tc)
            outfile3 = "results/expression/report/isoform_based/%s.%s.min_read_2_min_tc_%d.annotated.tsv" % (strain, group, tc)

            with open(outfile1, "w+") as fw:
                for run, cell in d[["Run", "Cell"]].values:
                    path = "results/expression/quant_isoforms/min_read_2_min_tc_%d/%s/%s.tsv" % (tc, run, cell)
                    fw.write(path + "\n")
            cmd = "./scripts/merge_counts.py %s %s" % (outfile1, outfile2)
            subprocess.check_call(cmd, shell=True)
            dat = pd.read_csv(outfile2, sep="\t", header=0, index_col=0)
            dat["NTR"] = dat["Nascent"] / dat["Total"]
            dat["TPM"] = dat["Total"] * 1e6 / sum(dat["Total"])
            dat["T"] = -3/np.log2(1-dat["NTR"])
            dat = dat.merge(anno, left_index=True, right_index=True, how="left")
            dat.to_csv(outfile3, sep="\t")
            os.remove(outfile1)
            os.remove(outfile2)

K562	all_cells	369


/home/chenzonggui/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


K562	control_cells	190


/home/chenzonggui/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


K562	treatment_cells	105


...

/home/chenzonggui/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


mESC	control_cells	64


/home/chenzonggui/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


mESC	treatment_cells	110


/home/chenzonggui/miniconda3/envs/py36/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


# Gene expression (matrix)

In [6]:
for strain in strains:
    for group in ["all_cells", "control_cells", "treatment_cells"]:
        d = info
        d = d[(d["Strain"] == strain) & (d["Time"] == 3) & (d["ActD"].isna()) & (d["UMIs"] >= 5000)]
        
        if strain == "K562":
            if group == "all_cells":
                d = d[(d["s4U"] == 0) | (d["s4U"] == 50)]
            elif group == "control_cells":
                d = d[(d["s4U"] == 0)]
            elif group == "treatment_cells":
                d = d[(d["TCRatio"] >= 0.008) & (d["s4U"] == 50)]
            else:
                assert False
        elif strain == "mESC":
            if group == "all_cells":
                d = d[(d["s4U"] == 0) | (d["s4U"] == 400)]
            elif group == "control_cells":
                d = d[(d["s4U"] == 0)]
            elif group == "treatment_cells":
                d = d[(d["TCRatio"] >= 0.01) & (d["s4U"] == 400)]
            else:
                assert False
        else:
            assert False
                    
        print(strain, group, len(d), sep="\t")

        for tc in [1, 2]:
            outfile1 = "results/expression/report/gene_based_matrix/%s.%s.min_read_2_min_tc_%d.filelist.txt" % (strain, group, tc)
            outfile2 = "results/expression/report/gene_based_matrix/%s.%s.min_read_2_min_tc_%d.total.tsv" % (strain, group, tc)
            outfile3 = "results/expression/report/gene_based_matrix/%s.%s.min_read_2_min_tc_%d.nascent.tsv" % (strain, group, tc)
            with open(outfile1, "w+") as fw:
                for run, cell in d[["Run", "Cell"]].values:
                    path = "results/expression/quant_genes/min_read_2_min_tc_%d/%s/%s.tsv" % (tc, run, cell)
                    fw.write(path + "\n")
            cmd = "./scripts/make_count_matrix.py %s %s %s" % (outfile1, outfile2, outfile3)
            # print(cmd)
            subprocess.check_call(cmd, shell=True)
            os.remove(outfile1)

K562	all_cells	369
K562	control_cells	190
K562	treatment_cells	105
mESC	all_cells	176
mESC	control_cells	64
mESC	treatment_cells	110


# Isoform expression (matrix)

In [7]:
for strain in strains:
    for group in ["all_cells", "control_cells", "treatment_cells"]:
        d = info
        d = d[(d["Strain"] == strain) & (d["Time"] == 3) & (d["ActD"].isna()) & (d["UMIs"] >= 5000)]
        
        if strain == "K562":
            if group == "all_cells":
                d = d[(d["s4U"] == 0) | (d["s4U"] == 50)]
            elif group == "control_cells":
                d = d[(d["s4U"] == 0)]
            elif group == "treatment_cells":
                d = d[(d["TCRatio"] >= 0.008) & (d["s4U"] == 50)]
            else:
                assert False
        elif strain == "mESC":
            if group == "all_cells":
                d = d[(d["s4U"] == 0) | (d["s4U"] == 400)]
            elif group == "control_cells":
                d = d[(d["s4U"] == 0)]
            elif group == "treatment_cells":
                d = d[(d["TCRatio"] >= 0.01) & (d["s4U"] == 400)]
            else:
                assert False
        else:
            assert False
                    
        print(strain, group, len(d), sep="\t")

        for tc in [1, 2]:
            outfile1 = "results/expression/report/isoform_based_matrix/%s.%s.min_read_2_min_tc_%d.filelist.txt" % (strain, group, tc)
            outfile2 = "results/expression/report/isoform_based_matrix/%s.%s.min_read_2_min_tc_%d.total.tsv" % (strain, group, tc)
            outfile3 = "results/expression/report/isoform_based_matrix/%s.%s.min_read_2_min_tc_%d.nascent.tsv" % (strain, group, tc)
            with open(outfile1, "w+") as fw:
                for run, cell in d[["Run", "Cell"]].values:
                    path = "results/expression/quant_isoforms/min_read_2_min_tc_%d/%s/%s.tsv" % (tc, run, cell)
                    fw.write(path + "\n")
            cmd = "./scripts/make_count_matrix.py %s %s %s" % (outfile1, outfile2, outfile3)
            # print(cmd)
            subprocess.check_call(cmd, shell=True)
            os.remove(outfile1)

K562	all_cells	369
K562	control_cells	190
K562	treatment_cells	105
mESC	all_cells	176
mESC	control_cells	64
mESC	treatment_cells	110
